In [1]:
import json
import geopandas as gpd
import pickle as pkl

from pull_PW_data import pull_data
from shapely.geometry import shape


In [2]:
query_string = '''
    select b.id, b.display_address, b.zip, st_asgeojson(b.geometry) as geometry_json
    from buildings b
    inner join building_geographies bg
        on b.id = bg.building_id
    inner join geographies g
        on bg.geography_id = g.id
    where g.name = 'Hamilton County'
    --where state = 'OH'
    --where zip in ('45140','45238','45231','45211','45215','45230','45239','45244','45240','45236','45248','45247','45241','45242','45224','45255','45251','45030','45237','45212','45208','45227','45205','45219','45243','45202','45233','45220','45229','45002','45249','45223','45246','45213','45209','45206','45225','45214','45216','45232','45207','45204','45217','45226','45950','45252','45052','45218','45203','45041','45174','45001','45033','45111','45228','45051','45201','45210','45222','45221','45234','45235','45250','45254','45253','45258','45263','45262','45267','45264','45269','45268','45271','45270','45274','45273','45296','45280','45299','45298','45999')
'''

In [3]:
try:
    with open('hamilton1.pkl', 'rb') as f:
        cincy_shape = pkl.load(f)
except FileNotFoundError:
    cincy_shape = pull_data(query_string)
    with open('hamilton1.pkl', 'wb') as f:
        pkl.dump(cincy_shape, f)

100000 rows appended
200000 rows appended
300000 rows appended
321583 rows appended
321583 rows appended
final row count: 321583


In [4]:
try:
    with open('hamilton_shape1.pkl', 'rb') as f:
        cincy_shape = pkl.load(f)
except FileNotFoundError:
    cincy_shape = cincy_shape[cincy_shape.geometry_json.notnull()]
    # Transform normal df to geopandas df
    cincy_shape = gpd.GeoDataFrame(cincy_shape)
    # Transform geojson to shapes
    cincy_shape['geometry'] = [shape(json.loads(x)) for x in cincy_shape['geometry_json']]
    # Set geometry for geodf
    cincy_shape = cincy_shape.set_geometry('geometry')
    cincy_shape.set_crs(crs="EPSG:4326", inplace=True)
    with open('hamilton_shape1.pkl', 'wb') as f:
        pkl.dump(cincy_shape, f)

In [5]:
cincy_shape.to_file('hamilton_shape')

/Users/arnavmalhotra/anaconda3/envs/pluto/lib/python3.10/site-packages/geopandas/io/file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/var/folders/_3/v349phgn4ljd3jkf3ll9p2cm0000gp/T/ipykernel_8583/997108018.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  cincy_shape.to_file('hamilton_shape')


In [14]:
roads = cincy_shape[cincy_shape['id'].isin([4026699, 4020375, 4020409, 4215435])]

In [15]:
#from shapely import MultiPolygon, Polygon, LineString, MultiLineString
roads['area'] = [x.area for x in roads['geometry']]
roads['perimeter'] = [x.length for x in roads['geometry']]

/Users/arnavmalhotra/anaconda3/envs/pluto/lib/python3.10/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/arnavmalhotra/anaconda3/envs/pluto/lib/python3.10/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [18]:
roads['perimeter:area'] = roads['area']/roads['perimeter']

/Users/arnavmalhotra/anaconda3/envs/pluto/lib/python3.10/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [19]:
roads

,id,display_address,zip,geometry_json,geometry,area,perimeter,perimeter:area
233193,4215435,1797 Leona Drive,45238,"{""type"":""MultiPolygon"",""coordinates"":[[[[-84.6...","MULTIPOLYGON (((-84.61664 39.12457, -84.61665 ...",7.402677e-08,0.001345,0.000055
314815,4026699,3430 Robinet Drive,45238,"{""type"":""MultiPolygon"",""coordinates"":[[[[-84.6...","MULTIPOLYGON (((-84.61844 39.13396, -84.61865 ...",7.356227e-07,0.011364,0.000065
318090,4020375,3403 Anaconda Drive,45211,"{""type"":""MultiPolygon"",""coordinates"":[[[[-84.5...","MULTIPOLYGON (((-84.58098 39.15293, -84.58111 ...",3.718873e-07,0.005627,0.000066
318505,4020409,3384 Rodeo Court,45211,"{""type"":""MultiPolygon"",""coordinates"":[[[[-84.5...","MULTIPOLYGON (((-84.57742 39.15208, -84.57745 ...",3.377843e-07,0.004269,0.000079
